In [1]:
import pandas as pd
import random
import string

In [21]:
def generate_alphabet_tuples():
    return [(chr(i), []) for i in range(ord('a'), ord('z')+1)]


In [27]:
def generate_character_mapping():
    alphabet_tuples = generate_alphabet_tuples()
    for i in range(8):
        ascii_low = string.ascii_lowercase
        for t in alphabet_tuples:
            c =  random.choice(ascii_low)
            t[1].append(c)
            ascii_low =ascii_low.replace(c,'')
    
    
    return alphabet_tuples

In [28]:
alphabet_tuples = generate_character_mapping()

In [29]:
alphabet_tuples

[('a', ['d', 't', 'g', 'b', 's', 'h', 'v', 't']),
 ('b', ['z', 'd', 'r', 'f', 'o', 'd', 'x', 'e']),
 ('c', ['f', 'g', 'x', 'q', 'u', 'v', 'r', 'v']),
 ('d', ['s', 'z', 'o', 'u', 'i', 'a', 't', 'z']),
 ('e', ['p', 'y', 'j', 'v', 'l', 'o', 'l', 'q']),
 ('f', ['m', 'f', 'f', 'r', 'd', 'f', 'u', 'w']),
 ('g', ['i', 'o', 'b', 'e', 'p', 'p', 'd', 'p']),
 ('h', ['x', 'l', 'y', 'i', 'f', 'n', 'j', 'j']),
 ('i', ['g', 'q', 'p', 'd', 't', 'g', 'i', 'r']),
 ('j', ['t', 'h', 'l', 'a', 'h', 't', 'p', 'o']),
 ('k', ['j', 'r', 'm', 'm', 'v', 'w', 'y', 's']),
 ('l', ['e', 'x', 'q', 'j', 'c', 'c', 'o', 'd']),
 ('m', ['c', 'a', 'k', 'c', 'k', 'x', 'b', 'g']),
 ('n', ['y', 'c', 'u', 'n', 'r', 'z', 'h', 'k']),
 ('o', ['r', 'e', 'h', 'x', 'y', 'y', 'm', 'u']),
 ('p', ['v', 's', 't', 'w', 'j', 'i', 'w', 'm']),
 ('q', ['n', 'u', 'z', 'z', 'q', 'r', 'g', 'n']),
 ('r', ['q', 'p', 'i', 'y', 'z', 'q', 'e', 'b']),
 ('s', ['u', 'm', 'w', 's', 'b', 'm', 'f', 'h']),
 ('t', ['o', 'n', 'd', 'p', 'g', 'j', 'z', 'x']),


## change to dataframe 

In [2]:
def generate_alphabet_df():
    # Generate a dataframe with indexes from 'a' to 'z' and columns 'value_1' to 'value_8'
    alphabet_df = pd.DataFrame(index=[chr(i) for i in range(ord('a'), ord('d')+1)], 
                               columns=['column_'+str(i) for i in range(1,3)])
    alphabet_df.index.name = "carrier_letter"
    
    # Populate the DataFrame
    for each_column in alphabet_df.columns:
        ascii_low = string.ascii_lowercase
        for each_index in alphabet_df.index:
            c = random.choice(ascii_low)
            alphabet_df.loc[each_index, each_column] = c
            ascii_low = ascii_low.replace(c,'')
            
    alphabet_df.reset_index(level=0, inplace=True)
            
    return alphabet_df

In [3]:
alphabet_df = generate_alphabet_df()


In [4]:
print(alphabet_df)

  carrier_letter column_1 column_2
0              a        p        m
1              b        u        z
2              c        f        w
3              d        d        s


In [5]:
def shift_single_column(df, column_name, shift_value =1):
    column_to_shift = df[column_name]
    
    lost_values = column_to_shift[-shift_value:]
    
    shifted_column = df[column_name].shift(shift_value)
   
    df[column_name] = pd.concat([lost_values, shifted_column[shift_value:]]).reset_index(drop=True)

    return df


In [6]:
alphabet_df_shifted = shift_single_column(alphabet_df, 'column_1')

In [7]:
alphabet_df_shifted

,carrier_letter,column_1,column_2
0,a,d,m
1,b,p,z
2,c,u,w
3,d,f,s


## shift columns to achieve no repetition in rows

In [21]:
data = {
    'carrier_letter': ['a', 'b', 'c', 'd'],
    'column_1': ['i', 'z', 'l', 'v'],
    'column_2': ['i', 'l', 'w', 'r'],
}
df = pd.DataFrame(data)

In [16]:
df

,carrier_letter,column_1,column_2
0,a,i,w
1,b,z,r
2,c,l,i
3,d,v,l


In [22]:
def fix_repetition(df):
    rows_iterator = 0
   
    while rows_iterator < len(df):
        display(df)
        if df['column_1'][rows_iterator] == df['column_2'][rows_iterator]:
            df = shift_single_column(df, 'column_2')
        rows_iterator += 1
        
   
    
    return df    

In [23]:
df_fixed = fix_repetition(df)
df_fixed

,carrier_letter,column_1,column_2
0,a,i,i
1,b,z,l
2,c,l,w
3,d,v,r


,carrier_letter,column_1,column_2
0,a,i,r
1,b,z,i
2,c,l,l
3,d,v,w


,carrier_letter,column_1,column_2
0,a,i,r
1,b,z,i
2,c,l,l
3,d,v,w


,carrier_letter,column_1,column_2
0,a,i,w
1,b,z,r
2,c,l,i
3,d,v,l


,carrier_letter,column_1,column_2
0,a,i,w
1,b,z,r
2,c,l,i
3,d,v,l
